In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from qm9.data.dataset_class import ProcessedDataset
from qm9.data.prepare import prepare_dataset
from qm9.data.utils import _get_species

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datafiles = prepare_dataset(datadir="/home/gongjingjing/tmp/data",
                            dataset='qm9',
                            subset=None,
                            splits=None,
                            force_download=False)

datasets = {}
for split, datafile in datafiles.items():
    with np.load(datafile) as f:
        datasets[split] = {
            key: val
            for key, val in f.items()
        }

keys = [list(data.keys()) for data in datasets.values()]
assert all([key == keys[0]
            for key in keys]), 'Datasets must have same set of keys!'

In [23]:
datasets["train"]["positions"][0][:5]

array([[-1.2698136e-02,  1.0858041e+00,  8.0009960e-03],
       [ 2.1504159e-03, -6.0313176e-03,  1.9761203e-03],
       [ 1.0117308e+00,  1.4637512e+00,  2.7657481e-04],
       [-5.4081506e-01,  1.4475266e+00, -8.7664372e-01],
       [-5.2381361e-01,  1.4379326e+00,  9.0639728e-01]], dtype=float32)

In [34]:
print(datasets["train"]['charges'][:2])
for k in datasets["train"]:
    print(k, datasets["train"][k].shape)

[[6 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [7 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
num_atoms (100000,)
charges (100000, 29)
positions (100000, 29, 3)
index (100000,)
A (100000,)
B (100000,)
C (100000,)
mu (100000,)
alpha (100000,)
homo (100000,)
lumo (100000,)
gap (100000,)
r2 (100000,)
zpve (100000,)
U0 (100000,)
U (100000,)
H (100000,)
G (100000,)
Cv (100000,)
omega1 (100000,)
zpve_thermo (100000,)
U0_thermo (100000,)
U_thermo (100000,)
H_thermo (100000,)
G_thermo (100000,)
Cv_thermo (100000,)


In [17]:
make_ajacency_matrix(5)

/tmp/ipykernel_1786773/3508418254.py:7: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  row = torch.range(0, n_nodes-1, dtype=torch.long).reshape(1, -1, 1).repeat(1, 1, n_nodes)
/tmp/ipykernel_1786773/3508418254.py:8: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  col = torch.range(0, n_nodes-1, dtype=torch.long).reshape(1, 1, -1).repeat(1, n_nodes, 1)


tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
        [1, 2, 3, 4, 0, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 4, 0, 1, 2, 3]])

In [1]:
from torch_geometric.datasets import QM9
from torch_geometric.data import Data
from torch_geometric.data import Batch
from torch_geometric.data import DataLoader
from torch_geometric.data.separate import separate
import torch

def make_adjacency_matrix(n_nodes):
    row = torch.arange(0, n_nodes, dtype=torch.long).reshape(1, -1, 1).repeat(1, 1, n_nodes)
    col = torch.arange(0, n_nodes, dtype=torch.long).reshape(1, 1, -1).repeat(1, n_nodes, 1)
    full_adj = torch.concat([row, col], dim=0).reshape(2, -1)
    diag_bool = torch.eye(n_nodes, dtype=torch.bool).reshape(-1)
    return full_adj[:, ~diag_bool]

def transform(data):
    data.x = data.x[:, :6]
    data.edge_index = make_adjacency_matrix(data.x.shape[0])
    data.edge_attr = None
    data.zx = torch.randn_like(data.x)
    data.zpos = torch.randn_like(data.pos)
    data.y = None
    data.z = None
    return data

ds = QM9(root="/home/gongjingjing/tmp/data/qm9", transform=transform)

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_ds = DataLoader(ds, batch_size=2, shuffle=True)
for _i, _d in enumerate(_ds):
    if _i > 4:
        break
    # separate(_d)
    print(_d)
    print(_d._slice_dict)
    print(_d._inc_dict)
    print(_d.ptr)
    # print(_d.name)

DataBatch(x=[33, 6], edge_index=[2, 512], pos=[33, 3], idx=[2], name=[2], zx=[33, 6], zpos=[33, 3], batch=[33], ptr=[3])
defaultdict(<class 'dict'>, {'x': tensor([ 0, 17, 33]), 'edge_index': tensor([  0, 272, 512]), 'pos': tensor([ 0, 17, 33]), 'idx': tensor([0, 1, 2]), 'name': tensor([0, 1, 2]), 'zx': tensor([ 0, 17, 33]), 'zpos': tensor([ 0, 17, 33])})
defaultdict(<class 'dict'>, {'x': tensor([0, 0]), 'edge_index': tensor([ 0, 17]), 'pos': tensor([0, 0]), 'idx': tensor([0, 0]), 'name': None, 'zx': tensor([0, 0]), 'zpos': tensor([0, 0])})
tensor([ 0, 17, 33])
DataBatch(x=[36, 6], edge_index=[2, 630], pos=[36, 3], idx=[2], name=[2], zx=[36, 6], zpos=[36, 3], batch=[36], ptr=[3])
defaultdict(<class 'dict'>, {'x': tensor([ 0, 15, 36]), 'edge_index': tensor([  0, 210, 630]), 'pos': tensor([ 0, 15, 36]), 'idx': tensor([0, 1, 2]), 'name': tensor([0, 1, 2]), 'zx': tensor([ 0, 15, 36]), 'zpos': tensor([ 0, 15, 36])})
defaultdict(<class 'dict'>, {'x': tensor([0, 0]), 'edge_index': tensor([ 0, 

/home/gongjingjing/.local/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [3]:
import torch
ch = torch.arange(20).reshape(4, 5)
torch.split(ch, [2, 3], dim=-1)

(tensor([[ 0,  1],
         [ 5,  6],
         [10, 11],
         [15, 16]]),
 tensor([[ 2,  3,  4],
         [ 7,  8,  9],
         [12, 13, 14],
         [17, 18, 19]]))

In [18]:
datasets["train"]["charges"].shape

torch.Size([100000, 29])

In [1]:
17*16+(44-17)*(44-17-1)

974